In [1]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
from tqdm import tqdm

import sys
sys.path.insert(0, '/home/mx85/Github/mitsuba3/build_stable/python')
sys.path.append('/home/mx85/Github/PyPackages')

import drjit as dr
import mitsuba as mi

mi.set_variant('cuda_ad_rgb')


from mitsuba import ScalarTransform4f as T
from bsdf import * 
from solid_texture_bsdf import * 
from spongecake_bsdf import * 
from utils import * 

In [2]:
cls_name = globals()['SurfaceBased']
feature_map_type = 'cloth'
cloth_type = 'plain_gap0075'
texture = 'id_map_lightblue.png'
mi.register_bsdf('spongecake_bsdf', lambda props: cls_name(props, \
        feature_map_type=feature_map_type, cloth_type = cloth_type, texture=texture, \
        delta_transmission=True))

In [3]:
# import scene and generate reference image
scene = mi.load_file('cloth/cloth_scene.xml')
params = mi.traverse(scene)

fiber = True
alpha = 0.3
optical_depth = 2.0
tiles = 128
specular_prob = 0.5

# # set other parameters
# params['bsdf-matpreview.surface_or_fiber'] = mi.Bool(fiber)
# params['bsdf-matpreview.alpha'] = mi.Float(alpha)
# params['bsdf-matpreview.optical_depth'] = mi.Float(optical_depth)
# params['bsdf-matpreview.tiles'] = mi.Float(tiles)
# params['bsdf-matpreview.specular_prob'] = mi.Float(specular_prob)
# params.update()
# print("params", params)

In [4]:
print("params", params)

params SceneParameters[
  ----------------------------------------------------------------------------------------
  Name                                 Flags    Type           Parent
  ----------------------------------------------------------------------------------------
  bsdf-matpreview.base_color.value     ∂        Color3f        SRGBReflectanceSpectrum
  bsdf-matpreview.alpha                ∂        Float          BSDF
  bsdf-matpreview.optical_depth        ∂        Float          BSDF
  bsdf-matpreview.tiles                ∂        Float          BSDF
  bsdf-matpreview.specular_prob        ∂        Float          BSDF
  bsdf-matpreview.surface_or_fiber              Bool           BSDF
  emitter-envmap.sampling_weight                float          EnvironmentMapEmitter
  emitter-envmap.scale                 ∂        Float          EnvironmentMapEmitter
  emitter-envmap.data                  ∂, D     TensorXf       EnvironmentMapEmitter
  emitter-envmap.to_world                 

In [7]:
save_to = 'ref.exr'
spp = 64
image = mi.render(scene, spp=spp)
ref = mi.util.write_bitmap(save_to, mi.Bitmap(image))

In [8]:
lr = 1e-3
opt = mi.ad.Adam(lr=lr)

In [9]:
# initialize the base_color to something else
params['bsdf-matpreview.base_color.value'] = mi.Color3f(0.5, 0.5, 0.5)
params.update()

[(SRGBReflectanceSpectrum[
    value = [[0.5, 0.5, 0.5]]
  ],
  {'value'}),
 (SurfaceBased[], {'base_color'}),
 (Scene[
    children = [
      SurfaceBased[],
      EnvironmentMapEmitter[
        filename = "envmap.exr",
        res = "[513, 256]",
        bsphere = BoundingSphere3fDC[
          center = [[0.025915, 0.372239, -0.00522423]],
          radius = [2.76019]
        ]
      ],
      PathIntegrator[
        max_depth = 8,
        rr_depth = 5
      ],
      PerspectiveCamera[
        x_fov = [39.5978],
        near_clip = 0.1,
        far_clip = 100,
        film = HDRFilm[
          size = [1920, 1080],
          crop_size = [1920, 1080],
          crop_offset = [0, 0],
          sample_border = 0,
          compensate = 0,
          filter = GaussianFilter[stddev=0.50, radius=2.00],
          file_format = OpenEXR,
          pixel_format = rgb,
          component_format = float16,
        ],
        sampler = IndependentSampler[
          base_seed = 0
          sample_cou